In [1]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd

In [3]:
@step
def importer() -> tuple[
    Annotated[np.array, "X_train"],
    Annotated[np.array, "X_test"],
    Annotated[np.array, "Y_train"],
    Annotated[np.array, "Y_test"],]:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, Y_train, Y_test = train_test_split(
        data, digits.target, test_size = 0.3, shuffle=False
    )

    return X_train, X_test, Y_train, Y_test

In [4]:
@step
def svc_trainer(
    X_train: np.ndarray,
    Y_train: np.ndarray
) -> ClassifierMixin:
    model = SVC(gamma=0.001)
    model.fit(X_train, Y_train)
    return model

In [5]:
@step
def evaluator(
    X_test: np.ndarray,
    Y_test: np.ndarray,
    model = ClassifierMixin
):
    test_acc = model.score(X_test,Y_test)
    print(f"Test accuracy : {test_acc}")


In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    X_train, X_test, Y_train, Y_test = importer()
    Model = svc_trainer(X_train=X_train, Y_train=Y_train)
    evaluator(X_test=X_test, Y_test=Y_test, model=Model)

In [7]:
digits_svc_pipline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 digits_svc_pipline = digits_pipeline()                                                       │
│   2                                                                                              │
│                                                                                                  │
│ /home/o/.pyenv/versions/3.10.13/lib/python3.10/site-packages/zenml/new/pipelines/pipeline.py:152 │
│ 6 in __call__                                                                                    │
│                                                                                                  │
│   1523 │   │   │   return self.entrypoint(*args, **kwargs)                                       │
│   1524 │   │                                                                                     │
│   1525 │   │   self.prepare(*args, **kwargs)                                                     │
│ ❱ 1526 │   │   return self._run(**self._run_args)                                                │
│   1527 │                                                                                         │
│   1528 │   def _call_entrypoint(self, *args: Any, **kwargs: Any) -> None:                        │
│   1529 │   │   """Calls the pipeline entrypoint function with the given arguments.               │
│                                                                                                  │
│ /home/o/.pyenv/versions/3.10.13/lib/python3.10/site-packages/zenml/new/pipelines/pipeline.py:625 │
│ in _run                                                                                          │
│                                                                                                  │
│    622 │   │   logger.info(f"Initiating a new run for the pipeline: `{self.name}`.")             │
│    623 │   │                                                                                     │
│    624 │   │   with track_handler(AnalyticsEvent.RUN_PIPELINE) as analytics_handler:             │
│ ❱  625 │   │   │   deployment, pipeline_spec, schedule, build = self._compile(                   │
│    626 │   │   │   │   config_path=config_path,                                                  │
│    627 │   │   │   │   run_name=run_name,                                                        │
│    628 │   │   │   │   enable_cache=enable_cache,                                                │
│                                                                                                  │
│ /home/o/.pyenv/versions/3.10.13/lib/python3.10/site-packages/zenml/new/pipelines/pipeline.py:116 │
│ 0 in _compile                                                                                    │
│                                                                                                  │
│   1157 │   │   # Update with the values in code so they take precedence                          │
│   1158 │   │   run_config = pydantic_utils.update_model(run_config, update=update)               │
│   1159 │   │                                                                                     │
│ ❱ 1160 │   │   deployment, pipeline_spec = Compiler().compile(                                   │
│   1161 │   │   │   pipeline=self,                                                                │
│   1162 │   │   │   stack=Client().active_stack,                                                  │
│   1163 │   │   │   run_configuration=run_config,                                                 │
│                                                                                                  │
│ /home/o/.pyenv/versions/3.10.13/lib/python3.10/site-package